<a href="https://colab.research.google.com/github/onakamari/gRNA_maker/blob/main/v1_gRNA_20210118.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Input: EnsembleのIDリスト(csv file)
Output: gRNA sequences (Google spreadsheet)

In [1]:
# Install Biopython package
!pip install Biopython

     |████████████████████████████████| 2.3MB 8.2MB/s 


In [2]:
#Export to google spreadsheet
from google.colab import auth

auth.authenticate_user()

import gspread

from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [3]:
#pandas and matplotlib.pyplot
import pandas as pd

import matplotlib.pyplot as plt

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1cdS-UhQN0x7timp3ogsVFh8LM2t0ZFKLUP-XLQ2nGCk/edit#gid=0')

In [4]:
sheet = wb.worksheet('muscle')
data = sheet.get_all_values()
#print(data)

df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:]

↑Ready　\
↓WIP

In [ ]:
#sequence checked ok 20210120
"""
from Bio.Seq import Seq

aa = ["ENSG00000271576", "ENSG00000214110","ENSG00000196205"]

for i in aa:
    seq_id =  i
    server = "https://rest.ensembl.org"
    ext = "/sequence/id/"+seq_id+"?"

    req = requests.get(server+ext, headers={"Content-Type" : "application/json"})

    if not req.ok:
      req.raise_for_status()
      sys.exit()

    decoded = req.json()
    m = Seq(decoded['seq'])
    c = m.complement()
    #print(repr(decoded['seq']))
    print(c)
"""

In [ ]:
#WIP seq retrieval from Ensembl
#Request to Ensembl API with multiple ids in a csv list
import requests, sys, csv

#import a list of accession number and save to a variable
csv_file = open("./only_mistaken_accession_number.csv", "r", encoding="ms932", errors="", newline="")
f = csv.reader(csv_file, delimiter=",", doublequote=True, lineterminator="\r\n", skipinitialspace=True)
header = next(f)
for row in f:
    r = row[0]
    print(r)

    seq_id =  r
    server = "https://rest.ensembl.org"
    ext = "/sequence/id/"+seq_id+"?"

    req = requests.get(server+ext, headers={"Content-Type" : "application/json"})

    if not req.ok:
      req.raise_for_status()
      sys.exit()

    decoded = req.json()
    print(repr(decoded['seq']))
    #save the response to a variable. Move on to the next step.




In [ ]:
raw_seq = []

#loop should start here!
seq_id =  
server = "https://rest.ensembl.org"
ext = "/sequence/id/"+seq_id+"?"

req = requests.get(server+ext, headers={"Content-Type" : "application/json"})

if not req.ok:
  req.raise_for_status()
  sys.exit()

decoded = req.json()
print(repr(decoded['seq']))
#save the response to a variable. Move on to the next step.

In [47]:
#Multiple data points OK!! 20210121
from Bio.Seq import Seq
import os, os.path, json, re
import pandas as pd

path_to_json = 'json/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if
              pos_json.endswith('.json')]
#print(json_files)

content = [] #Declare a variable to store the loop result.

for index, js in enumerate(json_files):
  with open(os.path.join(path_to_json,js)) as json_file:
    json_text = json.load(json_file)

    id = json_file.name[5:20] #WIP bad index for file name!!
    seq = json_text['seq']
    dotbracket = json_text['dotbracket']
    length = len(seq)
    single_strand = '........'  #Search parameter can be altered here. [single strand regions => 18] 20210125
    #single_strand = re.finditer(r'\.{28,}', json_text['seq']) #WIP in regular expression
    #single_strand = '............................' #single strand regions => 28
    #single_strand = '............................' #single strand regions => 27
    #single_strand = '............................' #single strand regions => 26
    #single_strand = '...........................' #single strand regions => 25
    #single_strand = '..........................' #single strand regions => 24
    #single_strand = '.........................' #single strand regions => 23
    #single_strand = '........................' #single strand regions => 22
    #single_strand = '.......................' #single strand regions => 21
    #single_strand = '......................' #single strand regions => 20
    #single_strand = '.....................' #single strand regions => 19

    #print(len(single_strand))
    num_ssregions = dotbracket.count(single_strand)  #Number of matching regions.

    data = {'id':id,'num_ssregions':num_ssregions, 'seq':seq, 'dotbracket':dotbracket, 'length':length}
    content.append(data) #Success! A dic is made! 2021012


In [48]:
print(len(single_strand))

8


In [50]:
#Single strand 8ntで検索した場合
total = 0
for ssr in content:
  x = ssr['num_ssregions'] #number of matching regions
  l = ssr['length'] #total length of lncRNA
  total += x #total number of estimated gRNA

  print(x, total, l)




40 40 5354
36 76 2761
16 92 961
42 134 3705
30 164 3410
107 271 7890
40 311 6295
79 390 6054
6 396 775
113 509 9172
22 531 1389
42 573 3524
12 585 892
34 619 2907
24 643 4298
82 725 8431
46 771 4125
47 818 3715
39 857 3537
50 907 3586
30 937 2398
2 939 271
18 957 1166
47 1004 6099
9 1013 829
17 1030 907
20 1050 998
81 1131 6440
23 1154 1601
65 1219 6713
62 1281 4534
21 1302 1749
71 1373 7838
7 1380 631
27 1407 2597
7 1414 682


In [51]:
#Subtract those with 0 matching regions from the above list
no_matching = []
matching = []

#Sort lncRNAs by the existence of matching regions. If not, delete from the original list.
for n in content:
  if n['num_ssregions'] == 0:
    no_matching.append(n)
  else:
    matching.append(n)

#List of lncRNA with no matching single strand regions.
for nm in no_matching:
  print('no matching', nm)

#List of lncRNA with matching single strand regions.
for m in matching:
  print('matching', m)



matching {'id': 'ENSG00000229388', 'num_ssregions': 40, 'seq': 'UGCGCUUUUAACUGCUUGUGGUUGAGUGGCAGCAUGAGCGAGCUCUGCGUAUUCCAACCCGGCGUUAAGAAUCCUUCACUAGCUUUUCUUCCCGACGCUGCUUAUUUAUCUAUCGGUAGCAUGACGGGUAAUUGACGCCCUCGGAUGGCUUUGAGGUCGCCAUUUUUGGUUGGUGCCUUUGUCUUGGUGUCAGGACUGGGAUAUCAUCUGGGACACAUUACCCCACAGGUAUCCUGUUGGACCUCAACGAGGAAGUUGGCAACAGAGGAACCAGGAGAUGAGAGAGAUCGGUUGGUUGUGCCUGUGAACUGGCAAUUCAGUAGUAACAAUUGAGUGCUUACUGUUUGCCGGGAAUUGUGCUAAUCCGCUUACCUCCGUUACGCCAUUUAAUCAUUUAGAGGAGACCUGUGAGAGAUUAAGCAAAGUUCUAGCUAAUAAGUGCAGAGCCUCAGCUUGAACUCAGAACUCACUACUAAGCCAAAGCCUGUCCUGUAGCAUUCUCUAAUGCUUUCAGUUUUUCGGCUGUCAGAUGCUUGGUGAGAACUGCUGUUAUGUGGCUCACGCCUGUAAUCCCAGCACUUUGGGAGGCCGAGGAGGGUGGAUCACGAGGUCAAGAGUUCAAGACCAGUCUGGCCAAGAUGGUGAAACCCCGUCUCUACUAAAAAUACAAAAAAUUAGCCGGGCGUGGUGGCAUGCGCCUGUAAUCGCAGCUACUCCGGAGGCUGACGCAGAGAACUGCUUAAACCUGGAGGGGCGGAGGUUGCAGUGAGCCGAGAUCGCGCCACUGCACUCCAGCCUGGGCGACAGAGCGAGACUCUGUCUCAAAAAAAAAAAAAAGAGAACAGCUGUUAUUUUUUACUGUAUGCUGGAAUGUUAGGCAUGCAAGUAAGACCCAAUUCCAGCAUCUGUAAGGUUCACAGUAAGAUGGAGAAGAAC

In [ ]:
"""
import re

color_dic = {'yellow':'\033[43m', 'red':'\033[31m', 'blue':'\033[34m', 'end':'\033[0m'}
keyword = ['........']

def print_hl(text, keyword, color="yellow"):
  for kw in keyword:
    bef = kw
    aft = color_dic[color] + kw + color_dic["end"]
    text = re.sub(bef, aft, text)
  print(text)

for kk in content:
  print_hl(kk['dotbracket'], keyword)
"""

In [52]:
#WIP SS regionsを先に絞り込む作戦

#1. 8tn以上のSSがあるかどうか確認 
for i, rna in enumerate(content):

#2. Trueのものだけで新しく配列を作る。(ただし、そのときは検索条件が正確であることが一層大事。)
  matching = []

  if rna['num_ssregions'] > 0:
    l = rna
    matching.append(l)

print(matching)

[{'id': 'ENSG00000232926', 'num_ssregions': 7, 'seq': 'GUGGGCCACGGGAUUCGUGGGCAGAGGAAGGGCACCAGCUCUGUGUUCCUUGUGCACAUGAAGCAGGAGGGCUCAUAGGCCUGCAUGCUGUGGACUUAGCUGAGCGGCACAGCUACAUCAAGGGCACCUUGAAGGACAUGCCCCUGCUGAGAUGGUCUCCAGGAUCCACACCAGUUUAAGCAGCAGACAGAGCUGUCCAUCACCACCAAGGGCCUCCACACGGGCCAAUUUCUGUUUGUGUACUGUAGCAAGAGGGCCCAGCUGGACGUCGGCAGCGUCCUCCCUGCAGGCACCACACCUGUACCAUCAAGCGCUGCCUGGAGGAGAGGCUGAGACUGGAACUUGUUAGCCUGGGCCUUUGAGAACUACACCGCAGUCACCUCCCCUAGCCCGGAAACCAGGAAGACUUACAUGAAGCUGCCCUCCAAGAGGAUCAUCUCAGCCAACAGAGCUGUGGUCAAUGUGGGAGCCAGAGGUGGCCGAACUGACCAGCCAUCCUGAAGGCAGGCCAUGCCCACUACAUAUAAGGCAAAGAAGAGCUGCUGACCAUGGUGCAGAGUAUGGCCGCAAGCCCCAAGGAACGUCCCUUCAGAGAUGGCAACCACUGGCACGUUGGCGAACCUGCCACUGCAGAGAUGUCUCUGCUGGUAGCAAAGUGGGUCUCAUUGCUGCCCACUGGACU', 'dotbracket': '((((((....((.(((.(((((((.((.(((..((..((...(((((((((.(((.......))).)))))((((.((((((..(((((((.((((((((((.((......((((((((((((...)))))).(((...((((.((.((.(((((......((.......)).......(((((.......))))))))))..)).)).)))).)))((((((((........))))))))..))

In [ ]:
print(matching)

In [85]:
"""
#pandas and matplotlib.pyplot
import pandas as pd
import matplotlib.pyplot as plt

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1cdS-UhQN0x7timp3ogsVFh8LM2t0ZFKLUP-XLQ2nGCk/edit#gid=0')

#Export to google spreadsheet
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())
"""

#gRNA generation with a single dic! 全配列分のgRNA集合リスト
from Bio.Seq import Seq
import math

#Pick up one RNA from "content" as "rna" which is a dictionary
for i, rna in enumerate(content):

  if rna['num_ssregions'] == 0:
    continue  
    
  grnas = [] 

  rna_name = rna['id']
  rna_seq = rna['seq']
  rna_total_length = rna['length']
  rna_dotbracket = rna['dotbracket']

  for m in re.finditer(r'\.{8,}', rna_dotbracket):
    dot_index = m.span() #m(0) index is retrieved. 
    #print(dot_index)

    #find the center of the central seed region
    center_point = (dot_index[1] - dot_index[0]) / 2
    center_point = math.floor(center_point)
    center_index = dot_index[0] + center_point #count -4/+4 as central seed & count -14/+14 from this point as a complete gRNA seq.
    #print(center_point, center_index)

    #find central seed region
    cs_start = center_index - 4
    cs_end = center_index + 4
    central_seed = rna_seq[cs_start : cs_end]
    complement_central_seed = Seq(central_seed)
    gcentral_seed = complement_central_seed.complement() #[seq2]: complementary seq to seq1 
    gcentral_seq = str(gcentral_seed)

    #print(central_seed)

    #find a full gRNA region
    grna_start = center_index - 14
    grna_end = center_index + 14
    guide_seq = rna_seq[grna_start : grna_end]
    complement_guide_seq = Seq(guide_seq)
    grna_seq = complement_guide_seq.complement() #[seq2]: complementary seq to seq1 
    grna = str(grna_seq)

    index_length = len(m.group())
    #print(dot_index, index_length)
    #target = Seq(c_seq[dot_index[0]:dot_index[1]]) #[seq1]: target in the original RNA seq
    target = rna_seq[dot_index[0]:dot_index[1]]
    complement_target = Seq(target)
    central_seed_seq = complement_target.complement() #[seq2]: complementary seq to seq1 
    central_seq = str(central_seed_seq)

    grna = {'id': rna_name, '8nt central_seed':gcentral_seq, 'grna':grna, 'ss_length' : index_length, 'index' : dot_index, 'target_seq':target, 'total_length': rna_total_length}
    grnas.append(grna) #1RNA配列分完成
    
  print(grnas)




"""
  #WIP Big loop starts here.
  #Create a new worksheet to save gRNA data. Each RNA takes one sheet.
  newWorksheet = wb.add_worksheet(title=rna_name, rows="100", cols="10")

  #Select the worksheet
  wsh = wb.worksheet(rna_name)

  #Write header to the worksheet
  wsh.update_acell('A1','ID')
  #write a header as many as the seq needs.
  wsh.update_acell('B1','gRNA')
  wsh.update_acell('C1','length')
  wsh.update_acell('D1','start')
  wsh.update_acell('E1','end')
  wsh.update_acell('F1','target_sequence')

  #Loop through each lncRNA in the list. Result is one sheet containing all the gRNA seq for one lncRNA.
  for index, item in enumerate(grnas):

    #Split index to two columns. List couldn't be sent to spreadsheet....
    start = item['index'][0]
    end = item['index'][1]

    #add "nt" at the end of length for readability
    length_nt = str(item['grna_length']) + 'nt'

    #cell_num = ['A','B','C','D','E','F'] 

    A = 'A' + str(index + 2) 
    B = 'B' + str(index + 2) 
    C = 'C' + str(index + 2) 
    D = 'D' + str(index + 2) 
    E = 'E' + str(index + 2) 
    F = 'F' + str(index + 2)  

    #Write gRNA seq to the worksheet
    wsh.update_acell(A,item['id'])
    wsh.update_acell(B,item['grna_seq'])
    wsh.update_acell(C,length_nt)
    wsh.update_acell(D,start)
    wsh.update_acell(E,end)
    wsh.update_acell(F,item['target_seq'])

"""

[{'id': 'ENSG00000229388', '8nt central_seed': 'AAAAUUGA', 'grna': '', 'ss_length': 8, 'index': (5, 13), 'target_seq': 'UUUUAACU', 'total_length': 5354}, {'id': 'ENSG00000229388', '8nt central_seed': 'GAUCGAAA', 'grna': 'CUUAGGAAGUGAUCGAAAAGAAGGGCUG', 'ss_length': 8, 'index': (78, 86), 'target_seq': 'CUAGCUUU', 'total_length': 5354}, {'id': 'ENSG00000229388', '8nt central_seed': 'UGUGUAAU', 'grna': 'UAGUAGACCCUGUGUAAUGGGGUGUCCA', 'ss_length': 9, 'index': (212, 221), 'target_seq': 'ACACAUUAC', 'total_length': 5354}, {'id': 'ENSG00000229388', '8nt central_seed': 'AACACGAU', 'grna': 'AACGGCCCUUAACACGAUUAGGCGAAUG', 'ss_length': 15, 'index': (351, 366), 'target_seq': 'GAAUUGUGCUAAUCC', 'total_length': 5354}, {'id': 'ENSG00000229388', '8nt central_seed': 'AAAUUAGU', 'grna': 'GCAAUGCGGUAAAUUAGUAAAUCUCCUC', 'ss_length': 9, 'index': (385, 394), 'target_seq': 'UUUAAUCAU', 'total_length': 5354}, {'id': 'ENSG00000229388', '8nt central_seed': 'AUUAUUCA', 'grna': 'UUCAAGAUCGAUUAUUCACGUCUCGGAG', 'ss_

'\n  #WIP Big loop starts here.\n  #Create a new worksheet to save gRNA data. Each RNA takes one sheet.\n  newWorksheet = wb.add_worksheet(title=rna_name, rows="100", cols="10")\n\n  #Select the worksheet\n  wsh = wb.worksheet(rna_name)\n\n  #Write header to the worksheet\n  wsh.update_acell(\'A1\',\'ID\')\n  #write a header as many as the seq needs.\n  wsh.update_acell(\'B1\',\'gRNA\')\n  wsh.update_acell(\'C1\',\'length\')\n  wsh.update_acell(\'D1\',\'start\')\n  wsh.update_acell(\'E1\',\'end\')\n  wsh.update_acell(\'F1\',\'target_sequence\')\n\n  #Loop through each lncRNA in the list. Result is one sheet containing all the gRNA seq for one lncRNA.\n  for index, item in enumerate(grnas):\n\n    #Split index to two columns. List couldn\'t be sent to spreadsheet....\n    start = item[\'index\'][0]\n    end = item[\'index\'][1]\n\n    #add "nt" at the end of length for readability\n    length_nt = str(item[\'grna_length\']) + \'nt\'\n\n    #cell_num = [\'A\',\'B\',\'C\',\'D\',\'E\',\'F\

In [ ]:
print(content)

[{'id': 'ENSG00000270504', 'seq': 'GCCGCCCGGGCACCCCUCGUCUGCGGGCGCGGGGUCCAAGUGUCCCCUGUCCAGGGCGGGCAGAAAGGAGAAGUCGCAAUCUCGGUUGAAGUCUCUUCCCCGCCUCCUUCGUGCACUUCAAGGCUGCCUCGGCCCCGCGCCCCUCCCGCGUCCCCGGCCCCGCACCUUGCCGCAGCGGCCCCUGCGCCAGUUCGGACCGGUCCUCGGGAAAGGCGUCGCGCAGGCGCUGCCACAGGCGGCCCAGGUCCGCCAGGCUGCGGUGCAGGUAGAGCACGCUGCGGUCCGACCACUCCGUGCGGAUCUCGAAGAACUCCUCUUCGUCGCCGCGCCGGCUGACGAUGAGCCUGCGGAUGCCGUUCACCCAGCAGCCGCGCACGAACAUGUUCACGAGCGACGUGCCCUCAAACACCGCCGAGGCCAUGUCGCACGCAUGCCCCCGCCAAGGGCUCCCCAGCCCCGCCGCCCGCCCGCCCGCAGGAGGCGCGCCCCGGCCGGGGUCGUCCCGGGUCUGUCCGUGGCCGGGGUCGUCCGGGGUCGGCCCGUCACUCCAAGGAGGCUGCGUAUGGCCCGCGUUCGGGGCAGCGGGGCGGCGCGGCGGCGAGUGGCUCGGGCCGGCCGGGCGCGGCUUUAUAAGGCGCUUCCCAUCGCGGGCGGCGCGGUAGUCGACGUCGAGGUUAGGCCGCGUCCGCCUCCCGCCCGCCUCCCGCGGAACGGGCGCCGCGGGCCGCUCUGGAAAGGAAGAGGUCCGGCUGGCCGCGCCCCCGCCGCCUCCGCAGAGACCGCGCUGCCGCGGCCACGAGAACCCCCGGAGCCGGGAGCCGGGCCGAGUCCCAAGCGCUCGCCCCCGCCUCGCGCCCCCCGCGCCUCCUCCCUCGCGGCCCCGGAGCCGCAGGUCGGUGACUGGACCAGCCGGGAAGGGAGGCGGGGCGGGCCCUGCUUGCCAGGCGCUCGGCGCAGACUUUUUU

In [54]:
for i, rna in enumerate(content):
  print(rna)

{'id': 'ENSG00000229388', 'num_ssregions': 40, 'seq': 'UGCGCUUUUAACUGCUUGUGGUUGAGUGGCAGCAUGAGCGAGCUCUGCGUAUUCCAACCCGGCGUUAAGAAUCCUUCACUAGCUUUUCUUCCCGACGCUGCUUAUUUAUCUAUCGGUAGCAUGACGGGUAAUUGACGCCCUCGGAUGGCUUUGAGGUCGCCAUUUUUGGUUGGUGCCUUUGUCUUGGUGUCAGGACUGGGAUAUCAUCUGGGACACAUUACCCCACAGGUAUCCUGUUGGACCUCAACGAGGAAGUUGGCAACAGAGGAACCAGGAGAUGAGAGAGAUCGGUUGGUUGUGCCUGUGAACUGGCAAUUCAGUAGUAACAAUUGAGUGCUUACUGUUUGCCGGGAAUUGUGCUAAUCCGCUUACCUCCGUUACGCCAUUUAAUCAUUUAGAGGAGACCUGUGAGAGAUUAAGCAAAGUUCUAGCUAAUAAGUGCAGAGCCUCAGCUUGAACUCAGAACUCACUACUAAGCCAAAGCCUGUCCUGUAGCAUUCUCUAAUGCUUUCAGUUUUUCGGCUGUCAGAUGCUUGGUGAGAACUGCUGUUAUGUGGCUCACGCCUGUAAUCCCAGCACUUUGGGAGGCCGAGGAGGGUGGAUCACGAGGUCAAGAGUUCAAGACCAGUCUGGCCAAGAUGGUGAAACCCCGUCUCUACUAAAAAUACAAAAAAUUAGCCGGGCGUGGUGGCAUGCGCCUGUAAUCGCAGCUACUCCGGAGGCUGACGCAGAGAACUGCUUAAACCUGGAGGGGCGGAGGUUGCAGUGAGCCGAGAUCGCGCCACUGCACUCCAGCCUGGGCGACAGAGCGAGACUCUGUCUCAAAAAAAAAAAAAAGAGAACAGCUGUUAUUUUUUACUGUAUGCUGGAAUGUUAGGCAUGCAAGUAAGACCCAAUUCCAGCAUCUGUAAGGUUCACAGUAAGAUGGAGAAGAACCCACAAACA

In [ ]:
#gRNA generation with a single dic! 全配列分のgRNA集合リスト
from Bio.Seq import Seq

for i, rna in enumerate(content):

  grnas = [] 

  rna_name = rna['id']
  rna_seq = rna['seq']
  rna_dotbracket = rna['dotbracket']
  search_object = re.finditer(r'\.{28,}', rna_dotbracket)

  if search_object:
    for m in re.finditer(r'\.{28,}', rna_dotbracket):
      dot_index = m.span() #m(0) index is retrieved. 
      index_length = len(m.group())
      #target = Seq(c_seq[dot_index[0]:dot_index[1]]) #[seq1]: target in the original RNA seq
      target = rna_seq[dot_index[0]:dot_index[1]]
      complement_target = Seq(target)
      grna_seq = complement_target.complement() #[seq2]: complementary seq to seq1 
      grna_seq = str(grna_seq)

      grna = {'id': rna_name, 'grna_seq':grna_seq, 'grna_length' : index_length, 'index' : dot_index, 'target_seq':target,}
      grnas.append(grna) #1RNA配列分完成
    
    grnas.append(grna)
    print(grnas)


[{'id': 'ENSG00000271959', 'grna_seq': 'GCUGAACAGAAAAAAAAAAAAAAAAUUAAUUAAAAGUCGGU', 'grna_length': 41, 'index': (3127, 3168), 'target_seq': 'CGACUUGUCUUUUUUUUUUUUUUUUAAUUAAUUUUCAGCCA'}]
[{'id': 'ENSG00000229152', 'grna_seq': 'UUUUUUUUUUUUUUUUUUUUCUUUUCUUUUUGAUC', 'grna_length': 35, 'index': (402, 437), 'target_seq': 'AAAAAAAAAAAAAAAAAAAAGAAAAGAAAAACUAG'}, {'id': 'ENSG00000229152', 'grna_seq': 'UUGUAGAGAGUUUUAAUUUUUUUUUUUUUUUU', 'grna_length': 32, 'index': (1844, 1876), 'target_seq': 'AACAUCUCUCAAAAUUAAAAAAAAAAAAAAAA'}, {'id': 'ENSG00000229152', 'grna_seq': 'UUGUAGAGAGUUUUAAUUUUUUUUUUUUUUUU', 'grna_length': 32, 'index': (1844, 1876), 'target_seq': 'AACAUCUCUCAAAAUUAAAAAAAAAAAAAAAA'}]
[{'id': 'ENSG00000229152', 'grna_seq': 'UUGUAGAGAGUUUUAAUUUUUUUUUUUUUUUU', 'grna_length': 32, 'index': (1844, 1876), 'target_seq': 'AACAUCUCUCAAAAUUAAAAAAAAAAAAAAAA'}]
[{'id': 'ENSG00000229152', 'grna_seq': 'UUGUAGAGAGUUUUAAUUUUUUUUUUUUUUUU', 'grna_length': 32, 'index': (1844, 1876), 'target_seq': 'AACAUCUC

In [58]:
print(grnas)

[{'id': 'ENSG00000232926', 'grna_seq': 'GUUAAAGA', 'grna_length': 8, 'index': (225, 233), 'target_seq': 'CAAUUUCU', 'total_length': 682}, {'id': 'ENSG00000232926', 'grna_seq': 'UGGUGUGGACAUGGUAGUU', 'grna_length': 19, 'index': (291, 310), 'target_seq': 'ACCACACCUGUACCAUCAA', 'total_length': 682}, {'id': 'ENSG00000232926', 'grna_seq': 'CUGACCUUGAAC', 'grna_length': 12, 'index': (333, 345), 'target_seq': 'GACUGGAACUUG', 'total_length': 682}, {'id': 'ENSG00000232926', 'grna_seq': 'TTTCTTCT', 'grna_length': 8, 'index': (530, 538), 'target_seq': 'AAAGAAGA', 'total_length': 682}, {'id': 'ENSG00000232926', 'grna_seq': 'CGACUGGU', 'grna_length': 8, 'index': (541, 549), 'target_seq': 'GCUGACCA', 'total_length': 682}, {'id': 'ENSG00000232926', 'grna_seq': 'TTCGGGGTT', 'grna_length': 9, 'index': (568, 577), 'target_seq': 'AAGCCCCAA', 'total_length': 682}]


In [ ]:
""" Template for gRNA generation for a single lncRNA
#gRNA generation with a single dic! 1配列分のgRNA集合リスト
from Bio.Seq import Seq

grnas = [] 

c_name = content[2]['id']
c = content[2] #replace this line with the loop head.
c_seq = content[2]['seq']

for m in re.finditer(r'\.{8,}', c['dotbracket']):
  dot_index = m.span() #m(0) index is retrieved. 
  index_length = len(m.group())
  #target = Seq(c_seq[dot_index[0]:dot_index[1]]) #[seq1]: target in the original RNA seq
  target = c_seq[dot_index[0]:dot_index[1]]
  complement_target = Seq(target)
  grna_seq = complement_target.complement() #[seq2]: complementary seq to seq1 
  grna_seq = str(grna_seq)

  grna = {'id': c_name, 'grna_seq':grna_seq, 'grna_length' : index_length, 'index' : dot_index, 'target_seq':target,}
  grnas.append(grna) #1RNA配列分完成

"""

In [ ]:
print(grnas)

[{'id': 'ENSG00000271576', 'grna_seq': 'GGGGGCGGGCGGTGG', 'grna_length': 15, 'index': (0, 15), 'target_seq': 'CCCCCGCCCGCCACC'}, {'id': 'ENSG00000271576', 'grna_seq': 'CGCGGAAU', 'grna_length': 8, 'index': (201, 209), 'target_seq': 'GCGCCUUA'}, {'id': 'ENSG00000271576', 'grna_seq': 'AAAAGGCUG', 'grna_length': 9, 'index': (266, 275), 'target_seq': 'UUUUCCGAC'}, {'id': 'ENSG00000271576', 'grna_seq': 'AGGUCCGAGA', 'grna_length': 10, 'index': (293, 303), 'target_seq': 'UCCAGGCUCU'}, {'id': 'ENSG00000271576', 'grna_seq': 'GGGUUAUU', 'grna_length': 8, 'index': (468, 476), 'target_seq': 'CCCAAUAA'}, {'id': 'ENSG00000271576', 'grna_seq': 'AAAGGAAAA', 'grna_length': 9, 'index': (526, 535), 'target_seq': 'UUUCCUUUU'}, {'id': 'ENSG00000271576', 'grna_seq': 'UUGAUUUU', 'grna_length': 8, 'index': (660, 668), 'target_seq': 'AACUAAAA'}, {'id': 'ENSG00000271576', 'grna_seq': 'GCAUUUUUGU', 'grna_length': 10, 'index': (706, 716), 'target_seq': 'CGUAAAAACA'}, {'id': 'ENSG00000271576', 'grna_seq': 'UAACUU

In [ ]:
#Split index to two columns. List couldn't be sent to spreadsheet....
start = grnas[0]['index'][0]
end = grnas[0]['index'][1]
#print(start, end) 90 103

#add "nt" at the end of length for readability
length_nt = str(grnas[0]['grna_length']) + 'nt'
print(length_nt)

In [ ]:
#Write gRNA seq to the worksheet
print(grnas[0])
wsh.update_acell('A2',grnas[0]['id'])
wsh.update_acell('B2',grnas[0]['grna_seq'])
wsh.update_acell('C2',length_nt)
wsh.update_acell('D2',start)
wsh.update_acell('E2',end)
wsh.update_acell('F2',grnas[0]['target_seq'])

In [ ]:
for index, item in enumerate(grnas):
  print(item)

In [ ]:
#Print out results to Google spreadsheet
#for g in all_grnas:
""" 
#Create a new worksheet to save gRNA data. Each RNA takes one sheet.
sheettitle = g['id']
newWorksheet = wb.add_worksheet(title=sheettitle, rows="100", cols="10")

#Select the worksheet
wsh = wb.worksheet(sheettitle)

#Write header to the worksheet
wsh.update_acell('A1','ID')
#write a header as many as the seq needs.
wsh.update_acell('B1','gRNA')
wsh.update_acell('C1','length')
wsh.update_acell('D1','start')
wsh.update_acell('E1','end')
wsh.update_acell('F1','target_sequence')
"""

#Loop through each lncRNA in the list. Result is one sheet containing all the gRNA seq for one lncRNA.
for index, item in enumerate(grnas):

  #Split index to two columns. List couldn't be sent to spreadsheet....
  start = item['index'][0]
  end = item['index'][1]

  #add "nt" at the end of length for readability
  length_nt = str(item['grna_length']) + 'nt'

  #raw starts from 2.
  #k = 2 #starts filling from "k2."
  #  while k < index: #excute until the last index is done.
  
  #cell_num = ['A','B','C','D','E','F'] 

  A = 'A' + str(index + 2) 
  B = 'B' + str(index + 2) 
  C = 'C' + str(index + 2) 
  D = 'D' + str(index + 2) 
  E = 'E' + str(index + 2) 
  F = 'F' + str(index + 2)  

  #Write gRNA seq to the worksheet
  wsh.update_acell(A,item['id'])
  wsh.update_acell(B,item['grna_seq'])
  wsh.update_acell(C,length_nt)
  wsh.update_acell(D,start)
  wsh.update_acell(E,end)
  wsh.update_acell(F,item['target_seq'])


[Notes]
Use the latest json files.

RNAfold again! 20210118
196205 OK
232926 OK
236618 OK
258056 OK
261737 OK
262855 OK
265298 OK
268518 OK
270175 OK 
271361 OK

